In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict, OrderedDict
import numpy as np
import seaborn as sns
%matplotlib inline

In [ ]:
def mean_signal_per_window(sites_signal, max_pos=1, window_size=1000):
    """
    mean signal in non-overlapping windows 
    of specified size
    
    sites_signal: list of tuples formatted like (site, signal)
    max_pos: the last chromosomal position at which an event was recorded 
    window_size: desired size of windows to bin signals
    """
    import numpy as np
    # generate windows of the specified size
    windows = np.arange(0, max_pos, window_size)
    window_counts = defaultdict(list)
    # loop over every methylated site and catalog its 
    # window number, along with the signal at that site
    for s in sites_signal:
        bin_index = int(np.digitize(s[0], windows))
        window_counts[bin_index * window_size].append(s[1])
    return sorted(window_counts.items())


In [ ]:
def convert_chrom_notation(chrom):
    """
    add 'chr' prefixes
    """
    if 'chr' in str(chrom):
        return str(chrom)
    else: return 'chr' + str(chrom)

## Read in the methylation and Red1 ChIP data

We'll need to make sure chromosome notation is constant in both files.

In [ ]:
# read in the Red1 data, and use new, simpler column names
red1 = pd.read_csv("red1-chip.tsv", sep="\t", header=0, 
                       names=["chrom", "pos", "background", "red1", "signal"])

# add chromosome prefixes
red1['chrom'] = red1['chrom'].apply(convert_chrom_notation)

# take a look at the data
red1.head()

In [ ]:
# read in the ONT methylation data
# rename "methylated_frequency" to "signal"
meth = pd.read_csv("methylation-frequency.tsv", sep='\t', header=0, 
                       names=["chrom", "pos", "end", "num_cpg_motifs", "depth", 
                              "methylated_reads", "signal", "sequence"])

meth.drop(columns=['end'], inplace=True)

# take a look at the data
meth.head()

## Plot the raw methylation and Red1 signals across the genome

This ends up being pretty messy!

In [ ]:
f, ax = plt.subplots(figsize=(20,10))

chrom = 'chr1'

meth_ = meth[meth['chrom'] == chrom].query('signal > 0.')
red1_ = red1[red1['chrom'] == chrom].query('signal > 0')

ax2 = ax.twinx()

sns.scatterplot(x="pos", y="signal", data=red1_, ax=ax, color='firebrick')
sns.scatterplot(x="pos", y="signal", data=meth_, ax=ax2, color='dodgerblue')

## Bin genome-wide signals into windows 

After binning the existing data into windows, we'll calculate the mean signal per window. This makes the plots much easier to parse, but it might be too simplistic to simply take the per-window mean.

In [ ]:
# create the figure object
f, ax = plt.subplots(figsize=(20,10))
ax2 = ax.twinx()

# the chromosome we want to look at
chrom = 'chr1'

# the desired window size
window_size = 500

# subset the data to exclude sites with zero signal
# also, exclude methylation sites with fewer than 10 reads
meth_filt = meth.query('signal > 0.')
red1_filt = red1.query('signal > 0')

def bin_data(df, chrom='chr8', window_size=1):
    """
    given a dataframe, subset to the chromosome of interest
    and calculate the median signal per window
    """
    # filter to only include the chromosome of interest
    df = df[df['chrom'] == chrom]
    # get arrays of positions and signals at those positions
    sites, signal = df['pos'].values, df['signal'].values
    # divide the data into `n_windows` and find the positions
    # that separate those windows (`edges`)
    max_pos = max(sites)    
    sites_signal = zip(sites, signal)
    # calculate the mean signal per window
    mean_signal = mean_signal_per_window(sites_signal, max_pos=max_pos, window_size=window_size)
    return [x[0] for x in mean_signal], [np.mean(x[1]) for x in mean_signal]

# get the bin coordinates and mean signal per bin
meth_x, meth_y = bin_data(meth_filt, chrom=chrom, window_size=window_size)
red1_x, red1_y = bin_data(red1_filt, chrom=chrom, window_size=window_size)

ax.plot(red1_x, red1_y, color="firebrick", label="ChIP", lw=2)
ax2.plot(meth_x, meth_y, color="dodgerblue", label="methylation", lw=2)

ax.legend(loc="upper left")
ax.set_ylabel("Mean ChIP signal")
ax2.legend(loc="upper right")
ax2.set_ylabel("Mean methylation frequency")